# 1. Update all data 


In [3]:
# %load ../src/data/get_data.py
import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json


def get_johns_hopkins():
    '''Get data by a git pull request, the source code has to be pulled first
        Result is stored in the predifined csv structure
    '''

    git_pull = subprocess.Popen("usr/bin/git pull",
                                cwd=os.path.dirname('../data/raw/COVID-19/'),
                                shell=True,
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE)

    (out, error) = git_pull.communicate()

    print("Error : " + str(error))
    print("out : " + str(out))


def get_current_data_germany():
    '''Get current data from germany, attention API endpoint not too stable
        Result data frame is stored as pd.DataFrame
    '''

    data = requests.get(
        'https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')
    json_object = json.loads(data.content)
    full_list = []
    for pos, each_dict in enumerate(json_object['features'][:]):
        full_list.append(each_dict['attributes'])

    pd_full_list = pd.DataFrame(full_list)
    pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv', sep=';')
    print(' Number of regions rows: '+str(pd_full_list.shape[0]))


if __name__ == '__main__':
    get_johns_hopkins()
    get_current_data_germany()


Error : b"'usr' is not recognized as an internal or external command,\r\noperable program or batch file.\r\n"
out : b''


KeyError: 'attributes'

# 2. Process Pipeline

In [ ]:
# %load ../src/data/process_JH_data.py
import pandas as pd
import numpy as np

from datetime import datetime


def store_relational_JH_data():
    data_path = '../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw = pd.read_csv(data_path)

    pd_data_base = pd_raw.rename(columns={
        'Country/Region': 'country',
        'Province/State': 'state'
    })
    pd_data_base['state'] = pd_data_base['state'].fillna('no')
    pd_data_base = pd_data_base.drop(['Lat', 'Long'], axis=1)

    pd_relational_model = pd_data_base.set_index(
        ['state',
         'country']).T.stack(level=[0, 1]).reset_index().rename(columns={
             'level_0': 'date',
             0: 'confirmed'
         })

    pd_relational_model['date'] = pd_relational_model.date.astype(
        'datetime64[ns]')

    pd_relational_model.to_csv(
        '../data/processed/COVID_relational_confirmed.csv', sep=';', index=False)
    print('Number of rows stored: '+str(pd_relational_model.shape[0]))


if __name__ == '__main__':

    store_relational_JH_data()


Number of rows stored: 250800


# 3. Filter and Doubling Rate Calculation 

In [5]:
# %load ../src/features/build_features.py
import numpy as np 
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_rate_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''
    
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)
    
    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_
    
    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)
        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result
        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result


def rolling_reg(df_input,col='confirmed'):
    ''' Rolling Regression to approximate the doubling time'
        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_rate_via_regression,raw=False)

    return result


def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame
        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame
        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['state','country']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


if __name__ == '__main__':
    test_data=np.array([2, 4, 6])
    result=get_rate_via_regression(test_data)
    print('the test slope is: '+str(result))


    pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()


    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')
    print(pd_result_larg.head())

the test slope is: [2.]
               date     state        country  confirmed  confirmed_filtered  \
NaN      2020-01-22       NaN    Afghanistan        0.0                 NaN   
116160.0 2020-01-22  Zhejiang         Cyprus        0.0                 0.0   
115280.0 2020-01-22  Zhejiang           Cuba        0.0                 0.0   
114400.0 2020-01-22  Zhejiang        Croatia        0.0                 0.0   
113520.0 2020-01-22  Zhejiang  Cote d'Ivoire        0.0                 0.0   

          confirmed_DR  confirmed_filtered_DR  
NaN                NaN                    NaN  
116160.0           NaN                    NaN  
115280.0           NaN                    NaN  
114400.0           NaN                    NaN  
113520.0           NaN                    NaN  


4. Visual Board